In [1]:
import sys
import os
try:
    import wget
except ModuleNotFoundError:
    !pip install wget
import wget
import re
try:
    from ufal.udpipe import Model, Pipeline
except ModuleNotFoundError:
    !pip install ufal.udpipe
from ufal.udpipe import Model, Pipeline
import numpy as np

!pip install progress
from progress.bar import IncrementalBar

In [2]:
# URL of the UDPipe model
# Модель для работы с тегами (Добавлять к словам тег, что это существительное, прилагательное и тд)
udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    wget.download(udpipe_model_url)
    print('Downloading done', file=sys.stderr)

In [3]:
print('\nLoading the model...', file=sys.stderr)
model = Model.load(udpipe_filename)
print('\nLoading done', file=sys.stderr)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')


Loading the model...

Loading done


In [4]:
def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text

In [5]:
def unify_sym(text):  # принимает строку в юникоде
    text = list_replace \
        ('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)

    text = list_replace \
        ('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)

    text = list_replace('\u2010\u2011', '\u002D', text)

    text = list_replace \
            (
            '\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000',
            '\u2002', text)

    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)

    text = list_replace \
            (
            '\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062',
            '.', text)

    text = list_replace('\u2217', '\u002A', text)

    text = list_replace('…', '...', text)

    text = list_replace('\u2241\u224B\u2E2F\u0483', '\u223D', text)

    text = list_replace('\u00C4', 'A', text)  # латинская
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)

    currencies = list \
            (
            '\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192'
        )

    alphabet = list \
            (
            '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!?~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ')

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = ''.join(cleaned_text)

    return cleaned_text

In [6]:
def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken

In [7]:
def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token

In [8]:
def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma


In [9]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    # Если частеречные тэги не нужны (например, их нет в модели), выставьте pos=False
    # в этом случае на выход будут поданы только леммы
    # По умолчанию знаки пунктуации вырезаются. Чтобы сохранить их, выставьте punct=True

    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    # если text == 'Я гулял с собакой, и случайно она укусила прохожего'
    # то processed будет равен следующему:
    """
      # newdoc
      # newpar
      # sent_id = 1
      # text = Я гулял с собакой, и случайно она укусила прохожего
      1	Я	я	PRON	_	Case=Nom|Number=Sing|Person=1	2	nsubj	_	_
      2	гулял	гулять	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
      3	с	с	ADP	_	_	4	case	_	_
      4	собакой	собака	NOUN	_	Animacy=Anim|Case=Ins|Gender=Fem|Number=Sing	2	obl	_	SpaceAfter=No
      5	,	,	PUNCT	_	_	4	punct	_	_
      6	и	и	CCONJ	_	_	9	cc	_	_
      7	случайно	случайно	ADV	_	Degree=Pos	9	advmod	_	_
      8	она	она	PRON	_	Case=Nom|Gender=Fem|Number=Sing|Person=3	9	nsubj	_	_
      9	укусила	укусить	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	2	conj	_	_
      10	прохожего	прохожий	ADJ	_	Animacy=Anim|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing	9	obj	_	SpacesAfter=\n
    """
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [10]:
def PreprocessingUpPipe(string: str, keep_pos=True, keep_punct=False):
  #print('Processing input...', file=sys.stderr)
  # strip() - удаляет пробелы с конца и с начала
  res = unify_sym(string.strip()) 
  output = process(process_pipeline, text=res, keep_pos=keep_pos, keep_punct=keep_punct)
  return output

In [11]:
print(PreprocessingUpPipe('Я гулял с собакой, и случайно она укусила прохожего'))

['я_PRON', 'гулять_VERB', 'с_ADP', 'собака_NOUN', 'и_CCONJ', 'случайно_ADV', 'она_PRON', 'укусить_VERB', 'прохожий_ADJ']


In [12]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [13]:
import zipfile

def CreatModel(model_url: str):
  m = wget.download(model_url)
  model_file = model_url.split('/')[-1]
  with zipfile.ZipFile(model_file, 'r') as archive:
      stream = archive.open('model.bin')
      return gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [81]:
model_nlp_url_ruscorpora_upos_cbow_300_20_2019 = 'http://vectors.nlpl.eu/repository/20/180.zip'
model_nlp_url_ruwikiruscorpora_upos_skipgram_300_2_2019 = 'http://vectors.nlpl.eu/repository/20/182.zip'
model_nlp_url_tayga_upos_skipgram_300_2_2019 = 'http://vectors.nlpl.eu/repository/20/185.zip'
model_nlp_url_ruwikiruscorpora_func_upos_skipgram_300_5_2019 = 'http://vectors.nlpl.eu/repository/20/183.zip'
model_nlp_url_news_upos_skipgram_300_5_2019 = 'http://vectors.nlpl.eu/repository/20/184.zip'



In [15]:
model_nlp_ruscorpora_upos_cbow_300_20_2019 = CreatModel(model_nlp_url_ruscorpora_upos_cbow_300_20_2019)


2020-06-01 01:48:12,868 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2020-06-01 01:48:23,313 : INFO : loaded (189193, 300) matrix from <zipfile.ZipExtFile [closed]>


In [16]:
model_nlp_ruwikiruscorpora_upos_skipgram_300_2_2019 = CreatModel(model_nlp_url_ruwikiruscorpora_upos_skipgram_300_2_2019)


2020-06-01 01:50:05,796 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2020-06-01 01:50:18,937 : INFO : loaded (248978, 300) matrix from <zipfile.ZipExtFile [closed]>


In [17]:
model_nlp_tayga_upos_skipgram_300_2_2019 = CreatModel(model_nlp_url_tayga_upos_skipgram_300_2_2019)


2020-06-01 01:52:34,088 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2020-06-01 01:52:46,984 : INFO : loaded (249565, 300) matrix from <zipfile.ZipExtFile [closed]>


In [18]:
model_nlp_ruwikiruscorpora_func_upos_skipgram_300_5_2019 = CreatModel(model_nlp_url_ruwikiruscorpora_func_upos_skipgram_300_5_2019)


2020-06-01 01:54:57,918 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2020-06-01 01:55:10,505 : INFO : loaded (248118, 300) matrix from <zipfile.ZipExtFile [closed]>


In [82]:
model_nlp_news_upos_skipgram_300_5_2019 = CreatModel(model_nlp_url_news_upos_skipgram_300_5_2019)


2020-06-01 08:02:26,199 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2020-06-01 08:02:40,574 : INFO : loaded (249318, 300) matrix from <zipfile.ZipExtFile [closed]>


In [19]:
print(model_nlp_ruscorpora_upos_cbow_300_20_2019.get_vector('день_NOUN').shape)

(300,)


In [20]:
!pip install nltk
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('russian'))

def filter_stop_words(string: str) -> str:
  word_tokens = word_tokenize(string)
  filtered_sentence = [w for w in word_tokens if w not in stop_words]
  ans = ' '.join(filtered_sentence)
  return ans

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arteeemik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/arteeemik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
url_vocabulary_positive_and_negarive_phrases = 'https://drive.google.com/file/d/1WwgzVl8J3gJv76NwICZsFf5CJi6vAMni'
name_file_vocabulary_positive_and_negarive_phrases = 'vocabulary_positive_and_negarive_phrases.txt'

url_cars_dataset_xml = 'https://drive.google.com/open?id=1FYaCRfSaWBeX-5SB8Zi9GYg1nmZH_A8N'
name_file_cars_dataset_xml = 'cars_dataset.xml'

In [22]:
if not os.path.isfile(name_file_vocabulary_positive_and_negarive_phrases):
  print('vocabulary_positive_and_negarive_phrases not found. Downloading...', file=sys.stderr)
  wget.download(url_vocabulary_positive_and_negarive_phrases, name_file_vocabulary_positive_and_negarive_phrases)
  print('Downloading done', file=sys.stderr)
else:
  print('vocabulary_positive_and_negarive_phrases alredy downloaded', file=sys.stderr)


if not os.path.isfile(name_file_cars_dataset_xml):
  print('cars_dataset_xml not found. Downloading...', file=sys.stderr)
  wget.download(url_cars_dataset_xml, name_file_cars_dataset_xml)
  print('Downloading done', file=sys.stderr)
else:
  print('cars_dataset_xml alredy downloaded', file=sys.stderr)


vocabulary_positive_and_negarive_phrases alredy downloaded
cars_dataset_xml alredy downloaded


In [23]:
run_senti_word = open(f'RuSentiLex2017_revised_new.txt', 'r')

In [24]:
negative = set()
positive = set()
for line in run_senti_word:
  if len(line) > 5 and line[0] != '!':
    sp = line.split(',')
    if line.find('negative') != -1:
      negative.add(sp[0])
    if line.find('positive') != -1:
      positive.add(sp[0])

negative = list(negative)
positive = list(positive)

In [25]:
print(f'len_negative : {len(negative)}, len_positive: {len(positive)}')

len_negative : 9801, len_positive: 3698


In [26]:
run_senti_word = open(f'RuSentiLex2017_revised_new.txt', 'r')
negative_one_word = set()
positive_one_word = set()
for line in run_senti_word:
  if len(line) > 3 and line[0] != '!':
    sp = line.split(',')
    if len(sp[0].split()) == 1:
        if line.find('negative') != -1:
          negative_one_word.add(sp[0])
        if line.find('positive') != -1:
          positive_one_word.add(sp[0])

negative_one_word = list(negative_one_word)
positive_one_word = list(positive_one_word)

In [27]:
print(f'len_negative_one_word : {len(negative_one_word)}, len_positive_one_word: {len(positive_one_word)}')



len_negative_one_word : 8113, len_positive_one_word: 2969


In [28]:
# Парсилка для xml файлов
import xml.etree.ElementTree as ET
import pandas as pd
  
class XML2DataFrame:
    def __init__(self, xml_data):
        self.root = ET.parse(xml_data).getroot()
    
    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]
    
    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed
    
    def data_to_pandas(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)


In [29]:
# Выкачиываем из гугл диска датасет с отзывами в формет xml и парсим его в пандас
cars_dataset_xml = open(f'cars_dataset.xml','r')
xmlcars_dataset_df = XML2DataFrame(f'cars_dataset.xml')
cars_dataset_pandas = xmlcars_dataset_df.data_to_pandas()


In [30]:
cars_dataset_pandas

,id,review,scores,safety,comfort,driveability,reliability,appearance,whole,text
0,985418,\n\t\t,\n\t\t\t,4,4,4,4,4,4,"Неплохой семейный авто, вместителен, удобен. Р..."
1,429257,\n\t\t,\n\t\t\t,3,3,4,4,4,4,На Мазде 626 я проехал полгода пока не продал ...
2,586065,\n\t\t,\n\t\t\t,5,5,5,5,5,5,Почти год езжу на этом авто машина 2007 года и...
3,627077,\n\t\t,\n\t\t\t,5,5,5,5,5,5,Автомобиль класс! Езжу 3 года ни каких нарекан...
4,1073854,\n\t\t,\n\t\t\t,5,5,5,5,5,5,Купил год назад б/у. Езжу много в день по 300 ...
...,...,...,...,...,...,...,...,...,...,...
8266,579246,\n\t\t,\n\t\t\t,5,5,5,4,5,4,"Доброго здоровья, уважаемые друзья и гости сай..."
8267,512078,\n\t\t,\n\t\t\t,5,5,5,5,5,5,Доброго всем здоровья! Сегодня я хочу рассказа...
8268,666156,\n\t\t,\n\t\t\t,5,5,4,5,5,5,Кроссоверы это наверно самый популярный класс ...
8269,694059,\n\t\t,\n\t\t\t,5,5,4,5,5,5,С наступающим Новым годом! Уважаемые друзья и ...


In [31]:
from tqdm.notebook import tqdm
statistics_word_in_pandas_cars = {}

for sentence in tqdm(cars_dataset_pandas['text'], desc='make_statistics_words_of_dataset'):
    sentence = PreprocessingUpPipe(sentence)
    for word_and_part_of_speech in sentence:
        if word_and_part_of_speech in statistics_word_in_pandas_cars:
            statistics_word_in_pandas_cars[word_and_part_of_speech] += 1
        else:
            statistics_word_in_pandas_cars[word_and_part_of_speech] = 1



In [32]:
# Получаем вектор предложения, которое передается через среднюю сумму всех векторов токенов этого предложения
# Будет использоваться для того, чтобы находить негативные фразы похожие положительные, и наоборот 
# Эта схема вроде неплохо себя показала на практике, что она сохраняет смысл предложения хорошо
def GetAverageVectorOfSentence(sentence: str, some_model):
  #sentence = filter_stop_words(sentence)
  tokens_sentence = PreprocessingUpPipe(sentence)
  cnt_tokens = len(tokens_sentence)

  vector_average = np.zeros(300)
  sz_for_averade = cnt_tokens

  if cnt_tokens == 0:
    return vector_average
  
  sz_for_averade = cnt_tokens
  for sen in tokens_sentence:
    if sen in some_model:
      vector_average += some_model.get_vector(sen)
    else:
      sz_for_averade = sz_for_averade - 1
  if sz_for_averade:
    vector_average = vector_average / sz_for_averade;
  if sz_for_averade == 0:
    vector_average = vector_average - 1000
  return vector_average

In [33]:
from typing import NamedTuple

class MapPhrases(NamedTuple):
    sen_from: str
    sen_to: str

def CreateSimilarMapping(
    voc_from: [str], 
    voc_to: [str], 
    some_model, 
    is_use_dictionary: bool,
    plus_word: [str],
    minus_word: [str],
    is_one_word: bool,
) -> {str: MapPhrases}:
  dict_similarity = {}

  array_vectors_to = np.zeros((len(voc_to), 300))
  array_split_sen_to = [''] * len(voc_to)

  plus_vector = np.zeros(300)
  minus_vector = np.zeros(300)
  for i in plus_word:
    plus_vector = plus_vector + GetAverageVectorOfSentence(i, some_model)
    
  for i in minus_word:
    minus_vector = minus_vector + GetAverageVectorOfSentence(i, some_model)
    
  for number, sen_to in enumerate(tqdm(voc_to, desc='creat_array_vectors_to')):
    array_vectors_to[number] = GetAverageVectorOfSentence(sen_to, some_model)
    array_split_sen_to[number] = PreprocessingUpPipe(sen_to)
    #if number < 20:
    #   print(f'similar: << {model_nlp.similar_by_vector(array_vectors_to[number])} >>')
    #   print(f'set_to: << {sen_to} >>, vector: << {array_vectors_to[number]} >>')

  for number, sen_from in enumerate(tqdm(voc_from, desc='similar sentences')):
    #sen_from = sen_from.lower()
    #sentence = filter_stop_words(sen_from)
    from_vector = GetAverageVectorOfSentence(sen_from, some_model)
    split_set_from = PreprocessingUpPipe(sen_from)
    
    if is_use_dictionary is False:
        from_vector = from_vector + plus_vector - minus_vector
    dict_similarity[split_set_from[0]] = MapPhrases(sen_from=split_set_from, sen_to=sen_from)
    #print(array_vectors_to)
    #model_nlp.most_similar_cosmul
    # косинусное произвездение фразы set_from с векторами фраз из списка voc_to
    vector_dist = some_model.cosine_similarities(from_vector, array_vectors_to)
    max_cos_dist = -10000000000;

    # Поиск ближайшей фразы к sen_from из списка фраз voc_to
    for number, sen_to in enumerate(voc_to):
      sen_to = sen_to.lower()
      #print(vector_dist[number])
      if vector_dist[number] > max_cos_dist \
        and (not is_one_word \
             or array_split_sen_to[number][0].split('_')[1] == split_set_from[0].split('_')[1]\
            and array_split_sen_to[number][0] in statistics_word_in_pandas_cars and \
            statistics_word_in_pandas_cars[array_split_sen_to[number][0]] > 10):
        max_cos_dist = vector_dist[number]
        dict_similarity[split_set_from[0]] = MapPhrases(sen_from=split_set_from, sen_to=sen_to)
    #else:
        #from_vector = from_vector + plus_vector - minus_vector
        #sen_to = (some_model.similar_by_vector(from_vector))[0][0].split('_')[0]
        #dict_similarity[split_set_from[0]] = MapPhrases(sen_from=sen_from, sen_to=sen_to)

  return dict_similarity

In [34]:
from dataclasses import dataclass

@dataclass
class ModelsMappingDict:
    negative_to_positive: {MapPhrases}
    positive_to_negative: {MapPhrases}

In [35]:
def CreateModelsMappingDict(
    negative: [str], 
    positive: [str], 
    model,
    is_use_dictionary=True,
    plus_word=['позитив', 'положительный', 'хороший'],
    minus_word=['негатив', 'отрицательный', 'плохой'],
    is_one_word=False,
) -> ModelsMappingDict:
    # Здесь мепятся негативные фразы с более похожими позитивными (через модели RusVectors)
    dict_negative_to_similar_positive = CreateSimilarMapping(
        negative, 
        positive, 
        model,
        is_use_dictionary,
        plus_word,
        minus_word,
        is_one_word,
    )
    
    # Здесь мепятся позитивные фразы с более похожими негативными (через модели RusVectors)
    dict_positive_to_similar_negative = CreateSimilarMapping(
        positive, 
        negative, 
        model,
        is_use_dictionary,
        minus_word,
        plus_word,
        is_one_word,
    )

    return ModelsMappingDict(
        negative_to_positive=dict_negative_to_similar_positive,
        positive_to_negative=dict_positive_to_similar_negative
    )
    

Создание словаря из негатива в позитив, и обратно, ища похожие слова на отрицательные в положительном словаре, и положительные слова в отрицательном словаре

In [36]:
dicts_ruscorpora_upos_cbow_300_20_2019 = CreateModelsMappingDict(
    negative,
    positive, 
    model_nlp_ruscorpora_upos_cbow_300_20_2019
)


In [37]:
dicts_ruscorpora_upos_cbow_300_20_2019_one_word = CreateModelsMappingDict(
    negative_one_word,
    positive_one_word, 
    model_nlp_ruscorpora_upos_cbow_300_20_2019,
    is_one_word=True,
)


In [38]:
dicts_ruwikiruscorpora_upos_skipgram_300_2_2019 = CreateModelsMappingDict(
    negative,
    positive, 
    model_nlp_ruwikiruscorpora_upos_skipgram_300_2_2019
)


In [39]:
dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word = CreateModelsMappingDict(
    negative_one_word,
    positive_one_word, 
    model_nlp_ruwikiruscorpora_upos_skipgram_300_2_2019,
    is_one_word=True,
)


In [40]:
dicts_tayga_upos_skipgram_300_2_2019 = CreateModelsMappingDict(
    negative,
    positive, 
    model_nlp_tayga_upos_skipgram_300_2_2019
)


In [41]:
dicts_tayga_upos_skipgram_300_2_2019_one_word = CreateModelsMappingDict(
    negative_one_word,
    positive_one_word, 
    model_nlp_tayga_upos_skipgram_300_2_2019,
    is_one_word=True,
)


In [42]:
dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019 = CreateModelsMappingDict(
    negative,
    positive, 
    model_nlp_ruwikiruscorpora_func_upos_skipgram_300_5_2019
)


In [86]:
dicts_nlp_news_upos_skipgram_300_5_2019 = CreateModelsMappingDict(
    negative,
    positive, 
    model_nlp_ruwikiruscorpora_func_upos_skipgram_300_5_2019
)

In [43]:
dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word = CreateModelsMappingDict(
    negative_one_word,
    positive_one_word, 
    model_nlp_ruwikiruscorpora_func_upos_skipgram_300_5_2019,
    is_one_word=True,
)

In [84]:

dicts_nlp_news_upos_skipgram_300_5_2019_one_word = CreateModelsMappingDict(
    negative_one_word,
    positive_one_word, 
    model_nlp_news_upos_skipgram_300_5_2019,
    is_one_word=True,
)

Создание словаря из негатива в позитив, и обратно, используя вычет и прибавления отрицательных и положительных векторов слов к нужному, для преобразования в нужный стиль

In [44]:
dicts_ruscorpora_upos_cbow_300_20_2019_using_arifm = CreateModelsMappingDict(
        negative,
        positive, 
        model_nlp_ruscorpora_upos_cbow_300_20_2019,
        is_use_dictionary=False
    )

In [45]:
dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm = CreateModelsMappingDict(
        negative,
        positive, 
        model_nlp_ruwikiruscorpora_upos_skipgram_300_2_2019,
        is_use_dictionary=False
    )

In [46]:
dicts_tayga_upos_skipgram_300_2_2019_using_arifm = CreateModelsMappingDict(
        negative,
        positive, 
        model_nlp_tayga_upos_skipgram_300_2_2019,
        is_use_dictionary=False
    )

In [47]:
dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm = CreateModelsMappingDict(
        negative,
        positive, 
        model_nlp_ruwikiruscorpora_func_upos_skipgram_300_5_2019,
        is_use_dictionary=False
    )

In [48]:
print(dicts_ruscorpora_upos_cbow_300_20_2019.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_ruwikiruscorpora_upos_skipgram_300_2_2019.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_tayga_upos_skipgram_300_2_2019.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])

print(dicts_ruscorpora_upos_cbow_300_20_2019.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_ruwikiruscorpora_upos_skipgram_300_2_2019.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_tayga_upos_skipgram_300_2_2019.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019.positive_to_negative[PreprocessingUpPipe('веселый')[0]])



MapPhrases(sen_from=['абортивный_ADJ'], sen_to='своеобразный')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='прогрессирование')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='адаптационный')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='прогрессирование')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='беспечный')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='беспечный')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='развеселый')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='беспечный')


In [49]:
print(dicts_ruscorpora_upos_cbow_300_20_2019_one_word.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_tayga_upos_skipgram_300_2_2019_one_word.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])

print(dicts_ruscorpora_upos_cbow_300_20_2019_one_word.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_tayga_upos_skipgram_300_2_2019_one_word.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word.positive_to_negative[PreprocessingUpPipe('веселый')[0]])



MapPhrases(sen_from=['абортивный_ADJ'], sen_to='своеобразный')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='адекватный')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='нестандартный')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='адекватный')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='шумный')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='грустный')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='грустный')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='шумный')


In [50]:
print(dicts_ruscorpora_upos_cbow_300_20_2019_one_word.positive_to_negative[PreprocessingUpPipe('неплохой')[0]])
print(dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word.positive_to_negative[PreprocessingUpPipe('неплохой')[0]])
print(dicts_tayga_upos_skipgram_300_2_2019_one_word.positive_to_negative[PreprocessingUpPipe('неплохой')[0]])
print(dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word.positive_to_negative[PreprocessingUpPipe('неплохой')[0]])



MapPhrases(sen_from=['неплохой_ADJ'], sen_to='плохой')
MapPhrases(sen_from=['неплохой_ADJ'], sen_to='посредственный')
MapPhrases(sen_from=['неплохой_ADJ'], sen_to='слабоватый')
MapPhrases(sen_from=['неплохой_ADJ'], sen_to='посредственный')


In [51]:
print(dicts_ruscorpora_upos_cbow_300_20_2019_using_arifm.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_tayga_upos_skipgram_300_2_2019_using_arifm.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])
print(dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm.negative_to_positive[PreprocessingUpPipe('абортивный')[0]])


print(dicts_ruscorpora_upos_cbow_300_20_2019_using_arifm.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_tayga_upos_skipgram_300_2_2019_using_arifm.positive_to_negative[PreprocessingUpPipe('веселый')[0]])
print(dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm.positive_to_negative[PreprocessingUpPipe('веселый')[0]])


MapPhrases(sen_from=['абортивный_ADJ'], sen_to='славный')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='позитив')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='позитив')
MapPhrases(sen_from=['абортивный_ADJ'], sen_to='позитив')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='негатив')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='негатив')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='скверный')
MapPhrases(sen_from=['веселый_ADJ'], sen_to='негатив')


In [56]:
# Замена в предложение всех высказываний из этого предложения(ключи), которые есть в словаре, на их значение в этом словаре
def ReplaceInSentenceUsingDictionary(
    sentence: str, 
    dictionary: dict, 
    rev_dictionary: dict
) -> (str, str, bool):
    
    sentence = sentence.lower()
    split_sentence = re.findall(r"[\w'-]+|[.,!?;:]",sentence)
    pre_split_sentence = [PreprocessingUpPipe(i) for i in split_sentence]
    for number, word in enumerate(pre_split_sentence):
        if len(word) > 0:
            pre_split_sentence[number] = word[0]
        else:
            pre_split_sentence[number] = ''
    split_original_sentence = split_sentence.copy()
    
    len_sentence = len(split_sentence)
    
    is_change_sentence = False
    
    for number, word in enumerate(split_sentence):
        pre_word = pre_split_sentence[number]
        if pre_word in dictionary and (number > 0 and split_sentence[number - 1] != 'не' and split_sentence[number - 1] != 'Не' or number == 0):
            split_sen_from = dictionary[pre_word].sen_from
            
            #split_sen_from = re.findall(r"[\w'-]+|[.,!?;:]",sen_from)
            if number + len(split_sen_from) <= len_sentence:
                check = True
                #print(f'number: {number}, split_sen_from: {split_sen_from},' 
                      #f'len(split_sen_from): {len(split_sen_from)},'
                      #f'sentence: {sentence}, len_sentence: {len_sentence}')
                for num, sen_from_word in enumerate(split_sen_from):
                    new_word = pre_split_sentence[number + num]
                    #print(new_word,split_sentence[number + num] )
                    if new_word != sen_from_word:
                        check = False
                if check:
                    is_change_sentence = True
                    for num, sen_from_word in enumerate(split_sen_from):
                        split_sentence[number + num] = ''
                        split_original_sentence[number + num] = split_original_sentence[number + num].upper()
                    split_sentence[number] = dictionary[pre_word].sen_to.upper()
        elif pre_word in rev_dictionary and (number > 0 and (split_sentence[number - 1] == 'не' or split_sentence[number - 1] == 'Не')):
            split_sen_from = rev_dictionary[pre_word].sen_from
            
            #split_sen_from = re.findall(r"[\w'-]+|[.,!?;:]",sen_from)
            if number + len(split_sen_from) <= len_sentence:
                check = True
                #print(f'number: {number}, split_sen_from: {split_sen_from},' 
                      #f'len(split_sen_from): {len(split_sen_from)},'
                      #f'sentence: {sentence}, len_sentence: {len_sentence}')
                for num, sen_from_word in enumerate(split_sen_from):
                    new_word = pre_split_sentence[number + num]
                    #print(new_word,split_sentence[number + num] )
                    if new_word != sen_from_word:
                        check = False
                if check:
                    is_change_sentence = True
                    for num, sen_from_word in enumerate(split_sen_from):
                        split_sentence[number + num] = ''
                        split_original_sentence[number + num] = split_original_sentence[number + num].upper()
                    split_sentence[number] = rev_dictionary[pre_word].sen_to.upper()
    
    sentence = ' '.join(split_sentence)
    original_sentence = ' '.join(split_original_sentence)
    return (original_sentence, sentence, is_change_sentence)



In [67]:
# Замена в отзывах нагативных слов на более похожие позитивные слова и выражения
def TrasformReviews(dataset, similaris_dict, rev_similaris_dict) -> ([str], [str], [bool]):
  refactor_reviews = []
  original_reviews = []
  is_change_reviews = []
  for review in tqdm(dataset, desc='TrasformReviews'):
    refactor_review = ReplaceInSentenceUsingDictionary(
        review, 
        similaris_dict,
        rev_similaris_dict,
    )
    
    if refactor_review[2]:
        original_reviews.append(refactor_review[0])
        refactor_reviews.append(refactor_review[1])
        is_change_reviews.append(refactor_review[2])
  return (original_reviews, refactor_reviews, is_change_reviews)



In [60]:
transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruscorpora_upos_cbow_300_20_2019.negative_to_positive,
    dicts_ruscorpora_upos_cbow_300_20_2019.positive_to_negative,
)

transformed_reviews_to_positive_ruwikiruscorpora_upos_skipgram_300_2_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019.negative_to_positive,
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019.positive_to_negative,
)

transformed_reviews_to_positive_tayga_upos_skipgram_300_2_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_tayga_upos_skipgram_300_2_2019.negative_to_positive,
    dicts_tayga_upos_skipgram_300_2_2019.positive_to_negative,
)

transformed_reviews_to_positive_ruwikiruscorpora_func_upos_skipgram_300_5_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019.negative_to_positive,
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019.positive_to_negative,
)



In [87]:
transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_nlp_news_upos_skipgram_300_5_2019.negative_to_positive,
    dicts_nlp_news_upos_skipgram_300_5_2019.positive_to_negative,
)


In [61]:
transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruscorpora_upos_cbow_300_20_2019_one_word.negative_to_positive,
    dicts_ruscorpora_upos_cbow_300_20_2019_one_word.positive_to_negative,
)

transformed_reviews_to_positive_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word.negative_to_positive,
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word.positive_to_negative,
)

transformed_reviews_to_positive_tayga_upos_skipgram_300_2_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_tayga_upos_skipgram_300_2_2019_one_word.negative_to_positive,
    dicts_tayga_upos_skipgram_300_2_2019_one_word.positive_to_negative,
)

transformed_reviews_to_positive_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word.negative_to_positive,
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word.positive_to_negative,
)


In [88]:
transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_nlp_news_upos_skipgram_300_5_2019_one_word.negative_to_positive,
    dicts_nlp_news_upos_skipgram_300_5_2019_one_word.positive_to_negative,
)

In [68]:
transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_using_arifm = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruscorpora_upos_cbow_300_20_2019_using_arifm.negative_to_positive,
    dicts_ruscorpora_upos_cbow_300_20_2019_using_arifm.positive_to_negative,
)

transformed_reviews_to_positive_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm.negative_to_positive,
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm.positive_to_negative,
)

transformed_reviews_to_positive_tayga_upos_skipgram_300_2_2019_using_arifm = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_tayga_upos_skipgram_300_2_2019_using_arifm.negative_to_positive,
    dicts_tayga_upos_skipgram_300_2_2019_using_arifm.positive_to_negative,
)

transformed_reviews_to_positive_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm.negative_to_positive,
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm.positive_to_negative,
)

In [69]:
transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruscorpora_upos_cbow_300_20_2019.positive_to_negative,
    dicts_ruscorpora_upos_cbow_300_20_2019.negative_to_positive,
)

transformed_reviews_to_negative_ruwikiruscorpora_upos_skipgram_300_2_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019.positive_to_negative,
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019.negative_to_positive,
)

transformed_reviews_to_negative_tayga_upos_skipgram_300_2_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_tayga_upos_skipgram_300_2_2019.positive_to_negative,
    dicts_tayga_upos_skipgram_300_2_2019.negative_to_positive,
)

transformed_reviews_to_negative_ruwikiruscorpora_func_upos_skipgram_300_5_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019.positive_to_negative,
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019.negative_to_positive,
)

In [89]:
transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019 = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_nlp_news_upos_skipgram_300_5_2019.positive_to_negative,
    dicts_nlp_news_upos_skipgram_300_5_2019.negative_to_positive,
)

In [70]:
transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruscorpora_upos_cbow_300_20_2019_one_word.positive_to_negative,
    dicts_ruscorpora_upos_cbow_300_20_2019_one_word.negative_to_positive,
)

transformed_reviews_to_negative_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word.positive_to_negative,
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word.negative_to_positive,
)

transformed_reviews_to_negative_tayga_upos_skipgram_300_2_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_tayga_upos_skipgram_300_2_2019_one_word.positive_to_negative,
    dicts_tayga_upos_skipgram_300_2_2019_one_word.negative_to_positive,
)

transformed_reviews_to_negative_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word.positive_to_negative,
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word.negative_to_positive,
)


In [90]:
transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019_one_word = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_nlp_news_upos_skipgram_300_5_2019_one_word.positive_to_negative,
    dicts_nlp_news_upos_skipgram_300_5_2019_one_word.negative_to_positive,
)

In [71]:
transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_using_arifm = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruscorpora_upos_cbow_300_20_2019_using_arifm.positive_to_negative,
    dicts_ruscorpora_upos_cbow_300_20_2019_using_arifm.negative_to_positive,
)

transformed_reviews_to_negative_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm.positive_to_negative,
    dicts_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm.negative_to_positive,
)

transformed_reviews_to_negative_tayga_upos_skipgram_300_2_2019_using_arifm = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_tayga_upos_skipgram_300_2_2019_using_arifm.positive_to_negative,
    dicts_tayga_upos_skipgram_300_2_2019_using_arifm.negative_to_positive,
)

transformed_reviews_to_negative_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm = TrasformReviews(
    cars_dataset_pandas['text'],
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm.positive_to_negative,
    dicts_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm.negative_to_positive,
)

In [91]:

df_transformed_to_negative = pd.DataFrame({
    'original_feedback': 
        [review for num, review in enumerate(transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019[0]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_to_negative_feedback_ruscorpora_upos_cbow_300_20_2019': [review for num, review in enumerate(transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_to_negative_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019': [review for num, review in enumerate(transformed_reviews_to_negative_ruwikiruscorpora_upos_skipgram_300_2_2019[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_to_negative_feedback_tayga_upos_skipgram_300_2_2019': [review for num, review in enumerate(transformed_reviews_to_negative_tayga_upos_skipgram_300_2_2019[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_to_negative_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019': [review for num, review in enumerate(transformed_reviews_to_negative_ruwikiruscorpora_func_upos_skipgram_300_5_2019[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019': [review for num, review in enumerate(transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019[1]) 
            if transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019[2][num]],
    
    
    
    'transformed_to_negative_feedback_ruscorpora_upos_cbow_300_20_2019_using_arifm': [review for num, review in enumerate(transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_using_arifm[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_using_arifm[2][num]],
    'transformed_to_negative_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm': [review for num, review in enumerate(transformed_reviews_to_negative_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_using_arifm[2][num]],
    'transformed_to_negative_feedback_tayga_upos_skipgram_300_2_2019_using_arifm': [review for num, review in enumerate(transformed_reviews_to_negative_tayga_upos_skipgram_300_2_2019_using_arifm[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_using_arifm[2][num]],
    'transformed_to_negative_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm': [review for num, review in enumerate(transformed_reviews_to_negative_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_using_arifm[2][num]],
 })

In [93]:
pd.set_option('max_colwidth', 500)

writer = pd.ExcelWriter('transformed_to_negative.xlsx', engine='xlsxwriter')
df_transformed_to_negative.to_excel(writer, 'Sheet1')
writer.save()

df_transformed_to_negative.head(50)
#compression_opts = dict(method='zip',о
                        #archive_name='df_transformed_to_negative.csv')  
#df_transformed_to_negative.to_csv('df_transformed_to_negative.zip', index=False,
          #compression=compression_opts)  

,original_feedback,transformed_to_negative_feedback_ruscorpora_upos_cbow_300_20_2019,transformed_to_negative_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019,transformed_to_negative_feedback_tayga_upos_skipgram_300_2_2019,transformed_to_negative_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019,transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019,transformed_to_negative_feedback_ruscorpora_upos_cbow_300_20_2019_using_arifm,transformed_to_negative_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm,transformed_to_negative_feedback_tayga_upos_skipgram_300_2_2019_using_arifm,transformed_to_negative_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm
0,"НЕПЛОХОЙ семейный авто , вместителен , УДОБЕН . расход топлива РАДУЕТ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : НЕПЛОХОЙ авто","НИЧЕГО ХОРОШЕГО семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ПЕЧАЛИТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : НИЧЕГО ХОРОШЕГО авто","НИЧЕГО ХОРОШЕГО семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива НЕ УДИВИТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : НИЧЕГО ХОРОШЕГО авто","НИЧЕГО ХОРОШЕГО семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ПЕЧАЛИТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : НИЧЕГО ХОРОШЕГО авто","МАЛО ХОРОШЕГО семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ВОЛНОВАТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : МАЛО ХОРОШЕГО авто","МАЛО ХОРОШЕГО семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ВОЛНОВАТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : МАЛО ХОРОШЕГО авто","НЕГАТИВ семейный авто , вместителен , НЕГАТИВ . расход топлива НЕГАТИВ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : НЕГАТИВ авто","ПЛОХОЙ семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива НЕГАТИВ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : ПЛОХОЙ авто","ПЛОХОЙ семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива УДРУЧАТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : ПЛОХОЙ авто","ПЛОХОЙ семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ОГОРЧАТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : ПЛОХОЙ авто"
1,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДВОДИЛ меня . ПРОЧНЫЙ кузов . машина просто КРАСАВИЦА . общее впечатление : НАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . РАСШАТЫВАТЬ кузов . машина просто АВАНТЮРИСТКА . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . ТЕСНЫЙ кузов . машина просто ГОРДЯЧКА . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . ХЛИПКИЙ кузов . машина просто ГОРДЯЧКА . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . ХРУПКИЙ кузов . машина просто ГОРДЯЧКА . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . ХРУПКИЙ кузов . машина просто ГОРДЯЧКА . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не СЛАВНЫЙ меня . НЕГАТИВ кузов . машина просто НЕГАТИВ . общее впечатление : НЕГАТИВ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОЗИТИВ меня . НЕГАТИВ кузов . машина просто НЕГАТИВ . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОЗИТИВ меня . НЕНАДЕЖНЫЙ кузов . машина просто УРОДИНА . общее вп

In [92]:

df_transformed_to_negative_one_word = pd.DataFrame({
    'original_feedback': 
        [review for num, review in enumerate(transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_one_word[0]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_to_negative_feedback_ruscorpora_upos_cbow_300_20_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_one_word[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_to_negative_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_negative_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_to_negative_feedback_tayga_upos_skipgram_300_2_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_negative_tayga_upos_skipgram_300_2_2019_one_word[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_to_negative_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_negative_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word[1]) 
            if transformed_reviews_to_negative_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019_one_word[1]) 
            if transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019_one_word[2][num]],
 })

In [94]:
pd.set_option('max_colwidth', 500)

writer = pd.ExcelWriter('transformed_to_negative_one_word.xlsx', engine='xlsxwriter')
df_transformed_to_negative_one_word.to_excel(writer, 'Sheet1')
writer.save()

df_transformed_to_negative_one_word.head(50)

,original_feedback,transformed_to_negative_feedback_ruscorpora_upos_cbow_300_20_2019_one_word,transformed_to_negative_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word,transformed_to_negative_feedback_tayga_upos_skipgram_300_2_2019_one_word,transformed_to_negative_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word,transformed_reviews_to_negative_nlp_news_upos_skipgram_300_5_2019_one_word
0,"НЕПЛОХОЙ семейный авто , вместителен , УДОБЕН . расход топлива РАДУЕТ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : НЕПЛОХОЙ авто","ПЛОХОЙ семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ВОЛНОВАТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : ПЛОХОЙ авто","ПОСРЕДСТВЕННЫЙ семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ВОЛНОВАТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : ПОСРЕДСТВЕННЫЙ авто","СЛАБОВАТЫЙ семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ОГОРЧАТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : СЛАБОВАТЫЙ авто","ПОСРЕДСТВЕННЫЙ семейный авто , вместителен , НЕУДОБНЫЙ . расход топлива ВОЛНОВАТЬ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : ПОСРЕДСТВЕННЫЙ авто","СРЕДНЕНЬКИЙ семейный авто , вместителен , ПРОСТЕНЬКИЙ . расход топлива ВИДИМЫЙ как и на всех японцах . автомат переключается без рывков . . . общее впечатление : СРЕДНЕНЬКИЙ авто"
1,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДВОДИЛ меня . ПРОЧНЫЙ кузов . машина просто КРАСАВИЦА . общее впечатление : НАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . СЛОЖНЫЙ кузов . машина просто УРОД . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . ТЕСНЫЙ кузов . машина просто УРОД . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . НЕНАДЕЖНЫЙ кузов . машина просто УРОД . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ПОДХОДИТЬ меня . ХРУПКИЙ кузов . машина просто КАПРИЗ . общее впечатление : НЕНАДЕЖНЫЙ автомобиль,на мазде 626 я проехал полгода пока не продал . за это время никогда не ОДОБРИТЬ меня . ТЕСНЫЙ кузов . машина просто ЗВЕРЬ . общее впечатление : НЕНАДЕЖНЫЙ автомобиль
2,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЁЗНОЙ ПРЕТЕНЗИИ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : ОТЛИЧНЫЙ автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : ПОСРЕДСТВЕННЫЙ автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : ПЛОХОЙ автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : ПЛОХОЙ автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : ПЛОХОЙ автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : СРЕДНЕНЬКИЙ автомобиль !
3,автомобиль КЛАСС ! езжу 3 года ни каких нареканий хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль ТИПА ! езжу 3 года ни каких нареканий хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль ТИПА ! езжу 3 года ни каких нареканий хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабоч

In [95]:
df_transformed_to_positive = pd.DataFrame({
    'original_feedback': 
        [review for num, review in enumerate(transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019[0]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_to_positive_feedback_ruscorpora_upos_cbow_300_20_2019': [review for num, review in enumerate(transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_to_positive_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019': [review for num, review in enumerate(transformed_reviews_to_positive_ruwikiruscorpora_upos_skipgram_300_2_2019[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_to_positive_feedback_tayga_upos_skipgram_300_2_2019': [review for num, review in enumerate(transformed_reviews_to_positive_tayga_upos_skipgram_300_2_2019[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_to_positive_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019': [review for num, review in enumerate(transformed_reviews_to_positive_ruwikiruscorpora_func_upos_skipgram_300_5_2019[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019[2][num]],
    'transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019': [review for num, review in enumerate(transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019[1]) 
            if transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019[2][num]],
        
    'transformed_to_positive_feedback_ruscorpora_upos_cbow_300_20_2019_using_arifm': [review for num, review in enumerate(transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_using_arifm[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_using_arifm[2][num]],
    'transformed_to_positive_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm': [review for num, review in enumerate(transformed_reviews_to_positive_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_using_arifm[2][num]],
    'transformed_to_positive_feedback_tayga_upos_skipgram_300_2_2019_using_arifm': [review for num, review in enumerate(transformed_reviews_to_positive_tayga_upos_skipgram_300_2_2019_using_arifm[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_using_arifm[2][num]],
    'transformed_to_positive_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm': [review for num, review in enumerate(transformed_reviews_to_positive_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_using_arifm[2][num]],
 })



In [96]:


writer = pd.ExcelWriter('transformed_to_positive.xlsx', engine='xlsxwriter')
df_transformed_to_positive.to_excel(writer, 'Sheet1')
writer.save()

df_transformed_to_positive.head(50)

,original_feedback,transformed_to_positive_feedback_ruscorpora_upos_cbow_300_20_2019,transformed_to_positive_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019,transformed_to_positive_feedback_tayga_upos_skipgram_300_2_2019,transformed_to_positive_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019,transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019,transformed_to_positive_feedback_ruscorpora_upos_cbow_300_20_2019_using_arifm,transformed_to_positive_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019_using_arifm,transformed_to_positive_feedback_tayga_upos_skipgram_300_2_2019_using_arifm,transformed_to_positive_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019_using_arifm
0,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИИ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной серьёзной ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !
1,автомобиль класс ! езжу 3 года ни каких НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких НИКАКИХ НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких НИКАКИХ НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких НИКАКИХ НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких БЕЗ НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких БЕЗ НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких НИКАКИХ НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких НИКАКИХ НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких НИКАКИХ НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких НИКАКИХ НАРЕКАН

In [97]:

df_transformed_to_positive_one_word = pd.DataFrame({
    'original_feedback': 
        [review for num, review in enumerate(transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_one_word[0]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
   
    'transformed_to_positive_feedback_ruscorpora_upos_cbow_300_20_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_one_word[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_to_positive_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_positive_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_to_positive_feedback_tayga_upos_skipgram_300_2_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_positive_tayga_upos_skipgram_300_2_2019_one_word[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_to_positive_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_positive_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word[1]) 
            if transformed_reviews_to_positive_ruscorpora_upos_cbow_300_20_2019_one_word[2][num]],
    'transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019_one_word': [review for num, review in enumerate(transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019_one_word[1]) 
            if transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019_one_word[2][num]],
 })




In [98]:

writer = pd.ExcelWriter('transformed_to_positive_one_word.xlsx', engine='xlsxwriter')
df_transformed_to_positive_one_word.to_excel(writer, 'Sheet1')
writer.save()

df_transformed_to_positive_one_word.head(50)

,original_feedback,transformed_to_positive_feedback_ruscorpora_upos_cbow_300_20_2019_one_word,transformed_to_positive_feedback_ruwikiruscorpora_upos_skipgram_300_2_2019_one_word,transformed_to_positive_feedback_tayga_upos_skipgram_300_2_2019_one_word,transformed_to_positive_feedback_ruwikiruscorpora_func_upos_skipgram_300_5_2019_one_word,transformed_reviews_to_positive_nlp_news_upos_skipgram_300_5_2019_one_word
0,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЁЗНОЙ ПРЕТЕНЗИИ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !,почти год езжу на этом авто машина 2007 года и пока нет ни одной СЕРЬЕЗНЫЙ ПРЕТЕНЗИЯ к этому авто ! ! ! всё устраивает ! ! ! общее впечатление : отличный автомобиль !
1,автомобиль класс ! езжу 3 года ни каких НАРЕКАНИЙ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких ПРЕТЕНЗИЯ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких ПРЕТЕНЗИЯ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких ПРЕТЕНЗИЯ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких ПРЕТЕНЗИЯ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !,автомобиль класс ! езжу 3 года ни каких ПРЕТЕНЗИЯ хоть авто уже 16 лет а он все бегает . все работает на ура . бензин ест мало общее впечатление : рабочий автомобиль !
2,"купил год назад б у . езжу много в день по 300 - 500 км . ПРОБЛЕМ не было , заливай бенз и меняй масло вовремя и все будет хорошо общее впечатление : хороший автомобиль","купил год назад б у . езжу много в день по 300 - 500 км . КРИЗИС не было , заливай бенз и меняй масло вовремя и все будет хорошо общее впечатление : хороший автомобиль","купил год назад б у . езжу много в день по 300 - 500 км . КРИЗИС не было , заливай бенз и меняй масло вовремя и все будет хорошо общее впечатление : хороший автомобиль","купил год назад б у . езжу много в день по 300 - 500 км . КРИЗИС не было , заливай бенз и меняй масло вовремя и все будет хорошо общее впечатление : хороший автомобиль","купил год назад б у . езжу много в день по 300 - 500 км . КРИЗИС не было , заливай бенз и меняй масло вовремя и все будет хорошо общее впечатление : хороший автомобиль","купил год назад б у . езжу много в день по 300 - 500 км . КРИЗИС не было , заливай бенз и меняй масло вовремя и все будет хорошо общее впечатление : хороший автомобиль"
3,"первая моя машина была она . очень мне нравилось в ней все . ЗАТРАТ никаких СЕРЬЕЗНЫХ не было , ездила более трех лет . общее впечатление : компактная , вместительная машинка !","первая моя машина была она . очень мне нравилось в ней все . ЭФФЕКТИВНОСТЬ никаких СЕРЬЕЗНЫЙ не было , ездила более трех лет . общее впечатление : компактная , вместительная машинка !","первая моя машина была она . очень мне нравилось в ней все . ЭФФЕКТИВНОСТЬ никаких СЕРЬЕЗНЫЙ не было , ездила более трех лет . общее впечатление : компактная , вместительная машинк

In [99]:
!pip install deeppavlov
!python -m deeppavlov install rusentiment_bert

from deeppavlov import build_model, configs



     |████████████████████████████████| 836 kB 670 kB/s eta 0:00:01
     |████████████████████████████████| 85 kB 4.6 MB/s eta 0:00:011
     |████████████████████████████████| 254 kB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 88 kB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 10.2 MB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 6.4 MB/s eta 0:00:011
     |████████████████████████████████| 10.5 MB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 859 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 15.1 MB 1.8 MB/s eta 0:00:011
     |████████████████████████████████| 8.0 MB

  Created wheel for httptools: filename=httptools-0.0.13-cp37-cp37m-macosx_10_9_x86_64.whl size=100554 sha256=a7fc66cdc1a59f3b403f866bb00aad7eaa49a774d32d4b4a3a78ce5c887f3340
  Stored in directory: /Users/arteeemik/Library/Caches/pip/wheels/4d/38/a5/408313d5d2f8abeb45df005a7780bd7e7a42bf6a46d434dd6a
  Created wheel for starlette: filename=starlette-0.12.9-py3-none-any.whl size=57242 sha256=0144b8daf01588f98b26850663c3507b4a5fa4607c29c19b7b264f20c8654a1f
  Stored in directory: /Users/arteeemik/Library/Caches/pip/wheels/e8/78/be/f57ed5aed7cd222abdb24e3186b5c9f1074184fcc0a295102b
Successfully built pytelegrambotapi overrides sacremoses httptools starlette
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.42.1
    Uninstalling tqdm-4.42.1:
      Successfully uninstalled tqdm-4.42.1
  Attempting uninstall: pytz
    Found existing installation: pytz 2019.3
    Uninstalling pytz-2019.3:
      Successfully uninstalled pytz-2019.3
  Attempting uninstall: numpy
    Found exist

In [100]:

model = build_model(configs.classifiers.rusentiment_bert, download=True)

model(['Я ненавижу тебя!', 'Я тебя обожаю!'])


2020-06-01 09:44:58.87 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /Users/arteeemik/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
2020-06-01 09:44:58,087 : INFO : Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /Users/arteeemik/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [01:39<00:00, 6.65MB/s] 
2020-06-01 09:46:37.840 INFO in 'deeppavlov.core.data.utils'['utils'] at line 242: Extracting /Users/arteeemik/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /Users/arteeemik/.deeppavlov/downloads/bert_models
2020-06-01 09:46:37,840 : INFO : Extracting /Users/arteeemik/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /Users/arteeemik/.deeppavlov/downloads/bert_models
2020-06-01 09:46:42.755 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Do

['negative', 'positive']

In [101]:

!pip install pandas
from pandas.io.common import _is_url
import pandas as pd

ImportError: cannot import name '_is_url' from 'pandas.io.common' (/opt/anaconda3/lib/python3.7/site-packages/pandas/io/common.py)

In [ ]:

# Выкачиываем из гугл диска датасет с отзывами в формет xml и парсим его в пандас
transformed_to_negative_one_word = pd.read_excel('transformed_to_negative_one_word.xlsx', sheet_name='Sheet1')
transformed_to_negative = pd.read_excel('transformed_to_negative.xlsx', sheet_name='Sheet1')
transformed_to_positive_one_word = pd.read_excel('transformed_to_positive_one_word.xlsx', sheet_name='Sheet1')
transformed_to_positive = pd.read_excel('transformed_to_positive.xlsx', sheet_name='Sheet1')


#transformed_to_negative_one_word.head()


In [ ]:
#transformed_to_negative_one_word
#transformed_to_negative
#transformed_to_positive_one_word
#transformed_to_positive

for i in transformed_to_negative_one_word:
  print(i)

In [ ]:
def check_sentences(first: str, second: str, from_, target):
  check = False
  for i in first:
    if i.isupper():
      check = True
      break
  if check is False:
    return (False, False)
  first = first.lower()
  second = second.lower()
  first = first + '.'
  second = second + '.'
  ans = model([first, second])
  if ans[0] == target:
    return (False, False)
  else:
    if ans[1] == target:
      return (True, True)
    else:
      return (False, True)

In [ ]:
def get_avarage_dict(my_pd, from_, target):
  ans_dict = {}
  original_feedback = 'original_feedback'
  for name in my_pd:
    #print(name)
    if name != original_feedback and name[0] != 'U':
      ans_dict[name] = [0, 0]

  for name in my_pd:
    if name != original_feedback and name[0] != 'U':
      for ind in tqdm(range(min(len(my_pd[original_feedback]), 1500))):
        #print(name, ind)
        origin_sentences = nltk.sent_tokenize(my_pd[original_feedback][ind])
        transformed_sentences = nltk.sent_tokenize(my_pd[name][ind])
        max_len = max(len(origin_sentences), len(transformed_sentences))
        for i in range(max_len):
          first, second = check_sentences(origin_sentences[i], transformed_sentences[i], from_, target)
          if second:
            if first:
              ans_dict[name][1] = ans_dict[name][1] + 1
            else:
              ans_dict[name][0] = ans_dict[name][0] + 1
  return ans_dict

In [ ]:
transformed_to_positive = get_avarage_dict(transformed_to_negative_one_word, 'negative', 'positive')
print(transformed_to_positive)

In [ ]:
transformed_to_positive_one_word = get_avarage_dict(transformed_to_negative_one_word, 'negative', 'positive')
print(transformed_to_positive_one_word)

In [ ]:
transformed_to_negative = get_avarage_dict(transformed_to_negative_one_word, 'positive', 'negative')
print(transformed_to_negative)

In [ ]:
#transformed_to_negative_one_word
#transformed_to_negative
#transformed_to_positive_one_word
#transformed_to_positive
#'negative', 'positive'
ans_transformed_to_negative_one_word = get_avarage_dict(transformed_to_negative_one_word, 'positive', 'negative')
print(ans_transformed_to_negative_one_word)